In [1]:
import numpy as np
import pandas as pd
import sklearn
%matplotlib inline
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz

C:\Program Files\Anaconda2\lib\site-packages\fuzzywuzzy-0.15.0-py2.7.egg\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning


## Train Data Features

In [54]:
# Loadin Train Data with basic features
train_data1 = pd.read_csv('train_data1.csv', sep=',')
train_data2 = pd.read_csv('train_data2.csv', sep=',', skiprows=1, header=None)

In [55]:
# Creating DF with features and duplicate variable only for final model
def append_df(df1, df2):
    # df1 with column names, and df2 without
    df2.columns = df1.columns
    df1 = df1.append(df2, ignore_index=True)
    return df1

In [56]:
train_data1 = append_df(train_data1,train_data2)
train_data1.shape

(404290, 21)

In [57]:
train_data1.count()

id                 404290
qid1               404290
qid2               404290
question1          404290
question2          404288
is_duplicate       404290
question1_form     404113
question2_form     404123
q1_form_len        404290
q2_form_len        404290
q1_length          404290
q2_length          404290
q1_unique          404290
q2_unique          404290
q1_form_uni        404290
q2_form_uni        404290
q1_form_char       404290
q2_form_char       404290
q1_q2_char_diff    404290
common_cnt         404290
prcnt_common       404218
dtype: int64

In [58]:
train_data1['prcnt_common'].fillna(train_data1['prcnt_common'].mean(), inplace=True)

In [59]:
train_data1['question1_form'].fillna('', inplace=True)
train_data1['question2_form'].fillna('', inplace=True)

In [67]:
train_data1.count()

id                 404290
qid1               404290
qid2               404290
question1          404290
question2          404288
is_duplicate       404290
question1_form     404290
question2_form     404290
q1_form_len        404290
q2_form_len        404290
q1_length          404290
q2_length          404290
q1_unique          404290
q2_unique          404290
q1_form_uni        404290
q2_form_uni        404290
q1_form_char       404290
q2_form_char       404290
q1_q2_char_diff    404290
common_cnt         404290
prcnt_common       404290
dtype: int64

In [71]:
#Empty cols in the training data
train_data1[(train_data1['question2'].isnull())]

,id,qid1,qid2,question1,question2,is_duplicate,question1_form,question2_form,q1_form_len,q2_form_len,...,q2_length,q1_unique,q2_unique,q1_form_uni,q2_form_uni,q1_form_char,q2_form_char,q1_q2_char_diff,common_cnt,prcnt_common
105780,105780,174363,174364,How can I develop android app?,NaN,0,develop android app,,3,0,...,0,6,0,3,0,17,0,17,0,0.0
201841,201841,303951,174364,How can I create an Android app?,NaN,0,creat android app,,3,0,...,0,7,0,3,0,15,0,15,0,0.0


In [73]:
train_data1.question2[(train_data1['question2'].isnull())]=''

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [77]:
def fuzzy_ratios(input_df):
    input_df['partial_ratio'] = input_df.apply(lambda row: fuzz.partial_ratio(row[u'question1_form'], row[u'question2_form']), axis=1)
    input_df['partial_token_set_ratio'] = input_df.apply(lambda row: fuzz.partial_token_set_ratio(row[u'question1_form'], row[u'question2_form']), axis=1)
    input_df['partial_token_sort_ratio'] = input_df.apply(lambda row: fuzz.partial_token_sort_ratio(row[u'question1_form'], row[u'question2_form']), axis=1)
    input_df['q_ratio'] = input_df.apply(lambda row: fuzz.QRatio(row[u'question1_form'], row[u'question2_form']), axis=1)
    input_df['token_set_ratio'] = input_df.apply(lambda row: fuzz.token_set_ratio(row[u'question1_form'], row[u'question2_form']), axis=1)
    input_df['token_sort_ratio'] = input_df.apply(lambda row: fuzz.token_sort_ratio(row[u'question1_form'], row[u'question2_form']), axis=1)
    input_df['w_ratio'] = input_df.apply(lambda row: fuzz.WRatio(row[u'question1_form'], row[u'question2_form']), axis=1)
#   Issue of the Non-ASCII Characters with the below two ratios, ignored as of now. Col_clean function for it, if to be used    
#    input_df['uw_ratio'] = input_df.apply(lambda row: fuzz.UWRatio(row[u'question1_form'], row[u'question2_form']), axis=1)
#    input_df['uq_ratio'] = input_df.apply(lambda row: fuzz.UQRatio(row[u'question1_form'], row[u'question2_form']), axis=1)
    return input_df

In [94]:
import re
# Removing the non-ascii characters
def col_clean(input_col):
    input_col['question1_form'] = input_col.apply(lambda row: re.sub( r'[^\x00-\x7F]+','', row[u'question1_form']), axis=1)
    input_col['question1_form'] = input_col.apply(lambda row: re.sub(r'(?:\s{2,})',' ', row[u'question1_form']), axis=1)
    input_col['question1_form'] = input_col.apply(lambda row: row[u'question1_form'].rstrip(), axis=1)
    input_col['question1_form'] = input_col.apply(lambda row: row[u'question1_form'].lstrip(), axis=1)
    return input_col

In [ ]:
# Not used as of now!!
#train_data1[['question1_form']]=col_clean(train_data1[['question1_form']])
#train_data1[['question2_form']]=col_clean(train_data1[['question2_form']])

In [161]:
train_data1.count()

id                          404290
qid1                        404290
qid2                        404290
question1                   404290
question2                   404290
is_duplicate                404290
question1_form              404290
question2_form              404290
q1_form_len                 404290
q2_form_len                 404290
q1_length                   404290
q2_length                   404290
q1_unique                   404290
q2_unique                   404290
q1_form_uni                 404290
q2_form_uni                 404290
q1_form_char                404290
q2_form_char                404290
q1_q2_char_diff             404290
common_cnt                  404290
prcnt_common                404290
partial_ratio               404290
partial_token_set_ratio     404290
partial_token_sort_ratio    404290
q_ratio                     404290
token_set_ratio             404290
token_sort_ratio            404290
w_ratio                     404290
dtype: int64

In [163]:
# Storing the file as csv for the XGB Model
train_data_save = train_data1
train_data_save.drop(['id','qid1','qid2','question1','question2'], axis=1, inplace=True)
train_data_save.to_csv('train_basic_fuzzy.csv',index=False)